In [1]:
#had a lot of issues trying to install the "imputeLCMD" package, so this was how I did it
# if (!requireNamespace("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install(c("impute", "pcaMethods"))

# install.packages("imputeLCMD")

In [2]:
setwd("/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.1. Data Imputation/Input")
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.1. Data Imputation/Output')
cur_date = "042522"

library(missForest)
library(imputeLCMD)
library(readxl)
library(tidyverse)
library(tmvtnorm)

#reading in files
ozone_data = data.frame(read_excel("Ozone_Sterol_Asthma_data_011822.xlsx", sheet = 2)) 

Loading required package: randomForest

randomForest 4.7-1

Type rfNews() to see new features/changes/bug fixes.

Loading required package: foreach

Loading required package: itertools

Loading required package: iterators

Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix

Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

Loading required package: pcaMethods

Loading required package: Biobase

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:randomForest’:

    combine


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grep

Goal is to generate (impute) values for missing data. Given that this dataset contains various categories (ie. cell differentials, cytokines) a different method will be employed for each.

> Lung Function Data: Random Forest (RF)

> Soluble Mediators (everything else): Quantile Regression Imputation of Left-Censored Data (QRILC)

**Also make note of subjects/ variables with missing data.

In [3]:
head(ozone_data)

,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,4,1,AS_1,AS,A,Post,NR,NR,Lung_Function,FEV1,4.3899999999999899
2,4,1,AS_1,AS,A,Pre,NR,NR,Lung_Function,FEV1,4.5999999999999899
3,4,1,AS_1,AS,A,Post,NR,NR,Lung_Function,FEV1,4.3899999999999899
4,4,1,AS_1,AS,A,Pre,NR,NR,Lung_Function,FEV1,NA
5,7,2,AS_2,AS,A,Post,R,NR,Lung_Function,FEV1,2.96
6,7,2,AS_2,AS,A,Pre,R,NR,Lung_Function,FEV1,3.08


In [4]:
#determing what subjects have missing data for all variables delinated based on the dfs the data will be split into
subject_presence_df = ozone_data %>%
    mutate(count = ifelse(Value != "NA", 1, 0)) %>%
    group_by(Category, Sample_Type, Time_Point, Subject_ID) %>%
    summarize(Subject_Presence_Percentage = sum(count)/ length(unique(Variable)) * 100) %>%
    arrange(Subject_Presence_Percentage)
head(subject_presence_df)

`summarise()` has grouped output by 'Category', 'Sample_Type', 'Time_Point'.
You can override using the `.groups` argument.


Category,Sample_Type,Time_Point,Subject_ID,Subject_Presence_Percentage
<chr>,<chr>,<chr>,<chr>,<dbl>
Cytokine,S,Pre,AS_3,0.00000
Sterol,S,Pre,AS_3,0.00000
Cytokine,P,Post,AS_1,96.77419
Cytokine,P,Post,AS_10,96.77419
Cytokine,P,Post,AS_11,96.77419
Cytokine,P,Post,AS_12,96.77419


In [5]:
#determining variables that have missing data for all variables delineated based on the dfs the data will be split into
variable_presence_df = ozone_data %>%
    mutate(count = ifelse(Value != "NA", 1, 0)) %>%
    group_by(Sample_Type, Time_Point, Variable) %>%
    #calculating the percentage of variables with data in all subjects
    summarize(Variable_Presence_Percentage = sum(count)/ length(unique(Subject_ID)) * 100) %>%
    arrange(Variable_Presence_Percentage)
head(variable_presence_df)

`summarise()` has grouped output by 'Sample_Type', 'Time_Point'. You can
override using the `.groups` argument.


Sample_Type,Time_Point,Variable,Variable_Presence_Percentage
<chr>,<chr>,<chr>,<dbl>
P,Post,Utero,0.00000
P,Pre,Utero,0.00000
S,Post,MPO,87.50000
S,Pre,MPO,87.50000
S,Pre,24OH_Chol,95.83333
S,Pre,25OH_Chol,95.83333


In [6]:
#implementing background filter where a variable needs to be present in at least 25% of subjects
#and where subject needs to have at least 25% of data for variables within a category/ time point/ sample
variable1_to_remove_df = ozone_data %>%
    filter(Sample_Type == "P" & Variable == "Utero")
variable2_to_remove_df = ozone_data %>%
    filter(Sample_Type == "S", Time_Point == "Pre", Category == "Cytokine", Subject_ID == "AS_3") 
variable3_to_remove_df = ozone_data %>%
    filter(Sample_Type == "S", Time_Point == "Pre", Category == "Sterol", Subject_ID == "AS_3") 
variables_to_remove_df = rbind(variable1_to_remove_df, variable2_to_remove_df, variable3_to_remove_df)

head(variables_to_remove_df)

,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,4,1,AS_1,AS,P,Post,NR,NR,Cytokine,Utero,NA
2,4,1,AS_1,AS,P,Pre,NR,NR,Cytokine,Utero,NA
3,7,2,AS_2,AS,P,Post,R,NR,Cytokine,Utero,NA
4,7,2,AS_2,AS,P,Pre,R,NR,Cytokine,Utero,NA
5,11,3,AS_3,AS,P,Post,NR,R,Cytokine,Utero,NA
6,11,3,AS_3,AS,P,Pre,NR,R,Cytokine,Utero,NA


In [7]:
#only keeping rows in ozone data that aren't in the other dataframe
ozone_data = anti_join(ozone_data, variables_to_remove_df[1:11]) %>%
    #making value col numeric instead of character
    mutate(Value = ifelse(Value == "NA", NA, as.numeric(Value))) 

head(ozone_data)

Joining, by = c("Original_Subject_No", "Subject_No", "Subject_ID",
"Disease_Status", "Sample_Type", "Time_Point", "Inflammatory_Responder",
"Lung_Responder", "Category", "Variable", "Value")
Warning message in ifelse(Value == "NA", NA, as.numeric(Value)):
“NAs introduced by coercion”


,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,4,1,AS_1,AS,A,Post,NR,NR,Lung_Function,FEV1,4.39
2,4,1,AS_1,AS,A,Pre,NR,NR,Lung_Function,FEV1,4.60
3,4,1,AS_1,AS,A,Post,NR,NR,Lung_Function,FEV1,4.39
4,4,1,AS_1,AS,A,Pre,NR,NR,Lung_Function,FEV1,NA
5,7,2,AS_2,AS,A,Post,R,NR,Lung_Function,FEV1,2.96
6,7,2,AS_2,AS,A,Pre,R,NR,Lung_Function,FEV1,3.08


In [8]:
#now splitting dataset based on category, sample, time point
preimputed_df = ozone_data %>%
    group_by(Category, Sample_Type, Time_Point) %>%
    group_split()

cell_differential_plasma_post_df = preimputed_df[[1]]
cell_differential_plasma_pre_df = preimputed_df[[2]]
cell_differential_sputum_post_df = preimputed_df[[3]]
cell_differential_sputum_pre_df = preimputed_df[[4]]
cytokine_plasma_post_df = preimputed_df[[5]]
cytokine_plasma_pre_df = preimputed_df[[6]]
cytokine_sputum_post_df = preimputed_df[[7]]
cytokine_sputum_pre_df = preimputed_df[[8]]
lung_function_airways_post_df = preimputed_df[[9]]
lung_function_airways_pre_df = preimputed_df[[10]]
sterol_plasma_post_df = preimputed_df[[11]]
sterol_plasma_pre_df = preimputed_df[[12]]
sterol_sputum_post_df = preimputed_df[[13]]
sterol_sputum_pre_df = preimputed_df[[14]]

# Random Forest Imputation

In [9]:
head(lung_function_airways_post_df)

Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
4,1,AS_1,AS,A,Post,NR,NR,Lung_Function,FEV1,4.39
4,1,AS_1,AS,A,Post,NR,NR,Lung_Function,FEV1,4.39
7,2,AS_2,AS,A,Post,R,NR,Lung_Function,FEV1,2.96
7,2,AS_2,AS,A,Post,R,NR,Lung_Function,FEV1,2.96
11,3,AS_3,AS,A,Post,NR,R,Lung_Function,FEV1,4.24
11,3,AS_3,AS,A,Post,NR,R,Lung_Function,FEV1,4.24


In [10]:
#imputing lung function data using RF
random_forest_imputation = function(dataset){
    wider_dataset = dataset %>%
        pivot_wider(names_from = Variable, values_from = Value)
    imputed_RF_object = missForest(as.matrix(wider_dataset[,10:14]))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(wider_dataset[,1:9], imputed_RF_df)) %>%
        pivot_longer(cols = 10:14, names_to = "Variable", values_to = "Value")
    
    return(imputed_final_df)
}

#calling fn
imputed_lung_function_airways_post_df = random_forest_imputation(lung_function_airways_post_df)
imputed_lung_function_airways_pre_df = random_forest_imputation(lung_function_airways_pre_df)

head(imputed_lung_function_airways_post_df)

Warning message:
“Values from `Value` are not uniquely identified; output will contain list-cols.
* Use `values_fn = list` to suppress this warning.
* Use `values_fn = {summary_fun}` to summarise duplicates.
* Use the following dplyr code to identify duplicates.
  {data} %>%
    dplyr::group_by(Original_Subject_No, Subject_No, Subject_ID, Disease_Status, Sample_Type, Time_Point, Inflammatory_Responder, Lung_Responder, Category, Variable) %>%
    dplyr::summarise(n = dplyr::n(), .groups = "drop") %>%
    dplyr::filter(n > 1L)”
Warning message in mean.default(xmis[, t.co], na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(xmis[, t.co], na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(xmis[, t.co], na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(xmis[, t.co], na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in m

  missForest iteration 1 in progress...done!


ERROR: Error in ximp[, t.ind] - ximp.old[, t.ind]: non-numeric argument to binary operator


# QRILC Imputation

In [ ]:
head(cell_differential_plasma_post_df)

In [ ]:
#imputing lung function data using RF
QRILC_imputation = function(dataset, number_of_variables){
    wider_dataset = dataset %>%
        pivot_wider(names_from = Variable, values_from = Value)
    
    #normalizing data since that what the QRILC function wants
    #had to pseudo log transform to prevent Inf values
    QRILC_prep = wider_dataset[,10:all_of(number_of_variables)] %>%
         mutate_all(., function(x) log10(x + 1)) %>%
         as.matrix()

    imputed_QRILC_object = impute.QRILC(QRILC_prep, tune.sigma = 0.1)
    QRILC_log10_df = data.frame(imputed_QRILC_object[1]) 
    
    #converting back the original scale
    QRILC_df = QRILC_log10_df %>%
        mutate_all(., function(x) 10^x - 1)
                   
    imputed_dataset = data.frame(cbind(dataset[,1:9], QRILC_df)) %>%
        pivot_longer(cols = 10:all_of(number_of_variables), names_to = "Variable", values_to = "Value")
    
    return(imputed_dataset)
}

#calling fn
imputed_cell_differential_plasma_post_df = QRILC_imputation(cell_differential_plasma_post_df, 24)
imputed_cell_differential_plasma_pre_df = QRILC_imputation(cell_differential_plasma_pre_df, 24)
imputed_cell_differential_sputum_post_df = QRILC_imputation(cell_differential_sputum_post_df, 24)
imputed_cell_differential_sputum_pre_df = QRILC_imputation(cell_differential_sputum_pre_df, 24)
imputed_cytokine_plasma_post_df = QRILC_imputation(cytokine_plasma_post_df, 39)
imputed_cytokine_plasma_pre_df = QRILC_imputation(cytokine_plasma_pre_df, 39)
imputed_cytokine_sputum_post_df = QRILC_imputation(cytokine_sputum_post_df, 39)
imputed_cytokine_sputum_pre_df = QRILC_imputation(cytokine_sputum_pre_df, 39)
imputed_sterol_plasma_post_df = QRILC_imputation(sterol_plasma_post_df, 27)
imputed_sterol_plasma_pre_df = QRILC_imputation(sterol_plasma_pre_df, 27)
# sterol_sputum_post_df had no missing data
# sterol_sputum_pre_df had no missing data

In [ ]:
#i think the data matrix doesn't like col names starting with numbers so it puts an X in front of it
# removing those X's prior to combining everything into 1 datafame
imputed_sterol_plasma_post_df = imputed_sterol_plasma_post_df %>%
     mutate(Variable = ifelse(grepl("X", Variable), 
                              unlist(strsplit(Variable, split = "X"))[2], Variable))
imputed_sterol_plasma_pre_df = imputed_sterol_plasma_pre_df %>%
    mutate(Variable = ifelse(grepl("X", Variable), 
                              unlist(strsplit(Variable, split = "X"))[2], Variable))

In [ ]:
#combining all imputed data
final_df = rbind(imputed_cell_differential_plasma_post_df, imputed_cell_differential_plasma_pre_df, imputed_cell_differential_sputum_post_df,
                imputed_cell_differential_sputum_pre_df, imputed_cytokine_plasma_post_df, imputed_cytokine_plasma_pre_df, imputed_cytokine_sputum_post_df,
                imputed_cytokine_sputum_pre_df, imputed_lung_function_airways_post_df, imputed_lung_function_airways_pre_df,
                imputed_sterol_plasma_post_df, imputed_sterol_plasma_pre_df, sterol_sputum_post_df,
                sterol_sputum_pre_df)
head(final_df)

In [ ]:
#exporting results
write.csv(final_df, paste0(Output,"/", cur_date, "_Imputed_Ozone_Data.csv"), row.names = FALSE)